# Very WIP!

Basic statistics - practical skills
- Maybe start with counting and summing?
- What is actually a...
    - mean, median, mode
    - standard deviation, standard error
    - median and quantile
- How to calculate them with pen & pape for small examples
- Appendix: how to calculate the standard quantities in Excel, Tableau, Python

How to calculate these quantities
- mean
- standard deviation
- standard error

[no, not this mean](https://en.wikipedia.org/wiki/File:Taylor_Swift_-_Mean.png#/media/File:Taylor_Swift_-_Mean.png)

if you fount this funny, you may like [this list.](https://www.isg.rhul.ac.uk/~sdg/mathsongs.html) You may also have issues.

In [1]:
import altair as alt
import helpers.plotting as pt
pt.enable_slide_theme()
pt.import_lato_font_in_notebook()

In [285]:
# test: interactive distribution parameters
plot_width=400

input_mean = alt.binding(
    input='range',
    min=-10,
    max=10,
    step=.1,
    name='mean'
)
input_std = alt.binding(
    input='range',
    min=0.1,
    max=5,
    step=.1,
    name='std'
)
mean_selection = alt.selection_single(
    bind=input_mean,
    init={'mean': 0}
)
std_selection = alt.selection_single(
    bind=input_std,
    init={'std': 1}
)
scales = alt.selection_interval(bind='scales')
                                
data = alt.sequence(0, 100, as_='t')
chart = alt.Chart(data).add_selection(
    scales, std_selection, mean_selection
).transform_calculate(
  mean = mean_selection['mean'],
  std  = std_selection['std'],
).transform_calculate(
    t='datum.t',
    x='sampleNormal(+datum.mean,+datum.std)', # the plus casts to number
)
scale = alt.Scale(domain=[-10,10])
hist = chart.mark_bar().encode(
    alt.X("x:Q", bin=True, scale=scale, axis=None),
    y='count()',
).properties(
    width=plot_width
)
ticks = chart.mark_tick().encode(
    x=alt.X('x:Q', scale=scale)
).properties(
    view=alt.ViewConfig(strokeWidth=0), width=plot_width)

(hist & ticks)

alt.VConcatChart(...)

In [1]:
import numpy as np
#from helpers.cards import draw_cards

In [2]:
stack1, stack2 = draw_cards(
    p_win_1 = 0.5,
    p_win_2 = 0.4,
    n_cards = 25,
    seed     = 0
)

In [3]:
m1 = stack1.mean()
m2 = stack2.mean()
n  = len(stack1)
np.sqrt(m1 * (1 - m1) / n), np.sqrt(m2 * (1 - m2) / n)

(0.09991996797437437, 0.09797958971132711)

In [4]:
np.sqrt(.5 * (1 - .5) / n), np.sqrt(.6 * (1 - .6) / n)

(0.1, 0.09797958971132711)